In [1]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import os
from pathlib import Path
import re
import pickle
# ver qtos o modelo apenas de ner acertaria
from transformers import AutoTokenizer, AutoModelForTokenClassification
import nltk    
from nltk import tokenize 
import torch
from transformers import BertTokenizer,BertForTokenClassification
import numpy as np
import json   
from importlib import reload  # Python 3.4+
import random
import model as mod
from model import BertForChunkClassification
from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup

from torch.utils.data import DataLoader
from importlib import reload 
#from eval import predict
import eval
#import importlib
#importlib.reload(module)
import dataset
from dataset import InputFeatures, load_and_cache_examples
import functionsAval as f

In [172]:
f = reload(f)
reload(dataset)
reload(eval)
reload(mod)

# em numero de frases
BATCH=100
#BATCH=5
#BATCH=800
#BATCH=8000 
print('BATCH:', BATCH)

BATCH: 100


In [107]:
dicSentences_new_test = f.loadSentencesTest()
print(len(dicSentences_new_test))
dicSentences_new_test = {k: v for k, v in dicSentences_new_test.items() if k<BATCH}
print(dicSentences_new_test[0])
#print(dicSentences_new_test[27])
print('numero de sentencas no total:', len(dicSentences_new_test))

sentences=list()
for key, value in dicSentences_new_test.items():
    if key<BATCH:
        tokens = value[0]
        tokens = [tok[0] for tok in tokens]
        sentences.append(' '.join(tokens).strip())
#print(sentences[0])

tags, tokens = f.predictBERTNER_IO(sentences, 'all')
dic_predictions = f.getDicPredictions(tags, tokens)
#print(dic_predictions[0])
print('len(dic_predictions):', len(dic_predictions))
#print(dic_predictions[9])
f.save_obj('dic_predictions_results_ner_'+str(BATCH), dic_predictions)
#dic_predictions = f.load_obj('dic_predictions_results_ner_'+str(BATCH))
print('verificando dados:')
#for key, value in dic_predictions.items():
#    print('key:',key)
#    print(dic_predictions[key])
#    if key>2:
#        break
        
print('len(dicSentences_new_test):', len(dicSentences_new_test))
print('len(dic_predictions):', len(dic_predictions))

region_true_list, region_pred_list, lista_erros = f.getListaRegionsTruePred(BATCH, dicSentences_new_test, dic_predictions)
f.save_obj('region_true_list'+str(BATCH), region_true_list)
print('region_pred_list[:4]:', region_pred_list[:4])
print('region_true_list[:4]:', region_true_list[:4])
print('lista_erros[:8]:', lista_erros[:8])
print('len(lista_erros):', len(lista_erros))
print('len(set(lista_erros)):', len(set(lista_erros)))
#print(dic_predictions[8])
#print(dicSentences_new_test[8][1])
print('len(region_true_list):', len(region_true_list))
print('len(region_pred_list):', len(region_pred_list))
#print('pred:',region_pred_list[:15])
#print('true:',region_true_list[:15])

print('-----Avaliando só modelo de NER:-----')

print(classification_report(region_true_list, region_pred_list, digits=6))
print(confusion_matrix(region_true_list, region_pred_list))


Pegando sentencas de teste gabarito: dic_sentencesTest.pkl
506
[[['Lucas', 0, 43], [',', 1, 48], ['74', 2, 50], ['anos', 3, 53], ['.', 4, 57]], []]
numero de sentencas no total: 100
idx2tag: {0: 'Teste', 1: 'Anatomia', 2: 'O', 3: 'Problema', 4: 'Tratamento', 5: '<pad>'}
len(dic_predictions): 100
verificando dados:
len(dicSentences_new_test): 100
len(dic_predictions): 100
region_pred_list[:4]: ['Problema', 'Tratamento', 'Problema', 'Problema']
region_true_list[:4]: ['Problema', 'Tratamento', 'Problema', 'Problema']
lista_erros[:8]: [7, 8, 13, 13, 14, 15, 15, 15]
len(lista_erros): 114
len(set(lista_erros)): 45
len(region_true_list): 352
len(region_pred_list): 352
-----Avaliando só modelo de NER:-----
              precision    recall  f1-score   support

    Anatomia   0.789474  0.277778  0.410959        54
           O   0.000000  0.000000  0.000000        38
    Problema   0.853211  0.823009  0.837838       113
       Teste   0.880952  0.891566  0.886228        83
  Tratamento   0.8281

### Testando modelo de sentence-pais
#### primeiro sozinho...
1) Com filtro + downsampling

2) Com filtro

3) Só positivos (dai preciso de um CRF pro filtro)

In [108]:
from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)
import pandas as pd
import logging

print('-----Avaliando só com modelo de Sentence Pairs:-----')
model = ClassificationModel('bert', 'lisaterumi/sentence_pairs_nested_filtro_downsampling', use_cuda=False)


-----Avaliando só com modelo de Sentence Pairs:-----


In [109]:
a=[0,0]
[num for num in range(a[0], a[1]+1, 1)]

[0]

In [198]:
f = reload(f)
#combinacaoEntidadesAll = f.getCombinacaoEntidadesSentence(dic_predictions, True, dicPosTagger, 0, lista_postaggers_entidades)
#combinacaoEntidadesAll

In [199]:
dic_predictions[20]

[[['Otimizo', 0],
  ['dose', 1],
  ['da', 2],
  ['sinvastatina', 3],
  ['para', 4],
  ['40mg', 5],
  ['/', 6],
  ['dia', 7],
  ['.', 8]],
 [['dose da sinvastatina para 40mg', [1, 2, 3, 4, 5], 'Tratamento']]]

In [200]:
df = pd.read_csv(r"..\preProcessamento\sentence-pairs-filtro\sentence_pairs.test", delimiter='\t', header=0, names=['text_b', 'text_a', 'labels'])
df = df.dropna(axis=0, how='any')
print('Número de sentenças de test: {:,}\n'.format(df.shape[0]))
#df=df[:50]
test_df = df
print(test_df[:3])
lista=list()
#for index, row in test_df.iterrows():
#    lista.append([row['text_b'], row['text_a']])
#predictions, raw_outputs = model.predict(
#  lista
#)
dic_sentencesTrainDev = f.load_obj('dic_sentencesTrainDev')
dicPosTagger, _ = f.getDicPosTagger(dic_sentencesTrainDev)
lista_postaggers_entidades = f.getListaPostaggerEntidades(dic_predictions, dicPosTagger)
# para gerar arquivo de predicoes (com as tags <e1>)
print('Primeiro, com filtro postagger:')
combinacaoEntidadesAll = f.getCombinacaoEntidadesSentence(dic_predictions, True, dicPosTagger, 0, lista_postaggers_entidades)
f.save_obj('combinacaoEntidadesAllSentence_'+str(BATCH), combinacaoEntidadesAll)



Número de sentenças de test: 1,913

         text_b                                             text_a  labels
0   marevan 5mg  Em acompanhamento no ambualtorio há 5 anos por...       2
1       marevan  Em acompanhamento no ambualtorio há 5 anos por...       0
2  Comorbidades  Comorbidades : DM há 10 anos em uso de metform...       1
Primeiro, com filtro postagger:
Sentence Pairs - Com filtro-postagger
Sentence Pairs - Sem taxa de Downsampling
erro_corpus: 0
num_frases_sem_entidade: 14
len(combinacaoEntidadesAll:) 100


In [201]:
combinacaoEntidadesAll[20]

[['dose da sinvastatina para 40mg',
  'Otimizo dose da sinvastatina para 40mg / dia .',
  [1, 2, 3, 4, 5],
  2],
 ['sinvastatina', 'Otimizo dose da sinvastatina para 40mg / dia .', [3], 0],
 ['dose', 'Otimizo dose da sinvastatina para 40mg / dia .', [1], 0],
 ['dose da sinvastatina',
  'Otimizo dose da sinvastatina para 40mg / dia .',
  [1, 2, 3],
  0],
 ['da', 'Otimizo dose da sinvastatina para 40mg / dia .', [2], 0],
 ['para', 'Otimizo dose da sinvastatina para 40mg / dia .', [4], 0],
 ['sinvastatina para 40mg',
  'Otimizo dose da sinvastatina para 40mg / dia .',
  [3, 4, 5],
  0]]

In [202]:
%%capture 
# para suprimir output
#combinacaoEntidadesAll = f.load_obj('combinacaoEntidadesAll_'+str(BATCH))
print('Chamando predict')
pred_region_labels = list()
for key, combinacao in enumerate(combinacaoEntidadesAll):
    if key<BATCH:    
        if len(combinacao)>0:
            lista = [l[0:2] for l in combinacao]
            predictions, _ = model.predict(lista) 
            pred_region_labels.append(predictions)
            for comb, label in zip(combinacao, predictions):
                comb.append(label)
f.save_obj('combinacaoEntidadesAllSentenceComLabel1_'+str(BATCH), combinacaoEntidadesAll)


In [114]:
labels = {0:'O', 1:'Problema', 2:'Tratamento', 3:'Teste', 4:'Anatomia'}
pred_region_labels2 = list()
for a in pred_region_labels:
    for b in a:
        pred_region_labels2.append(labels[b])


In [115]:
print(pred_region_labels2[:10])
print(region_true_list[:10])
print(len(pred_region_labels2))
print(len(region_true_list))

['Problema', 'Tratamento', 'Tratamento', 'Problema', 'Problema', 'Tratamento', 'Tratamento', 'Tratamento', 'Tratamento', 'O']
['Problema', 'Tratamento', 'Problema', 'Problema', 'Tratamento', 'Tratamento', 'Tratamento', 'Tratamento', 'Problema', 'Tratamento']
564
352


In [116]:
dic_predictions[3]

[[['HAS', 0],
  ['há', 1],
  ['15', 2],
  ['anos', 3],
  ['em', 4],
  ['uso', 5],
  ['de', 6],
  ['losartana', 7],
  ['50mg', 8],
  ['/', 9],
  ['dia', 10],
  ['e', 11],
  ['digoxina', 12],
  ['1', 13],
  ['/', 14],
  ['2', 15],
  ['cp', 16],
  ['/', 17],
  ['dia', 18],
  [',', 19],
  ['carvedilol', 20],
  ['25', 21],
  ['12', 22],
  ['/', 23],
  ['12', 24],
  [',', 25],
  ['HCTZ', 26],
  ['.', 27]],
 [['HAS', [0], 'Problema'],
  ['losartana 50mg', [7, 8], 'Tratamento'],
  ['digoxina', [12], 'Tratamento'],
  ['carvedilol 25', [20, 21], 'Tratamento'],
  ['HCTZ', [26], 'Tratamento']]]

In [203]:
combinacaoEntidadesAll[20]

[['dose da sinvastatina para 40mg',
  'Otimizo dose da sinvastatina para 40mg / dia .',
  [1, 2, 3, 4, 5],
  2,
  2],
 ['sinvastatina', 'Otimizo dose da sinvastatina para 40mg / dia .', [3], 0, 0],
 ['dose', 'Otimizo dose da sinvastatina para 40mg / dia .', [1], 0, 0],
 ['dose da sinvastatina',
  'Otimizo dose da sinvastatina para 40mg / dia .',
  [1, 2, 3],
  0,
  2],
 ['da', 'Otimizo dose da sinvastatina para 40mg / dia .', [2], 0, 0],
 ['para', 'Otimizo dose da sinvastatina para 40mg / dia .', [4], 0, 0],
 ['sinvastatina para 40mg',
  'Otimizo dose da sinvastatina para 40mg / dia .',
  [3, 4, 5],
  0,
  2]]

In [204]:
def getDicPredictionsSentence(combinacaoEntidadesAll_pred, dic_predictions):
    labels = {0:'O', 1:'Problema', 2:'Tratamento', 3:'Teste', 4:'Anatomia'}
    num=-1
    dic_predictions_sentence={}
    entidades=list()
    numAcrescentou=0
    for frases in combinacaoEntidadesAll_pred:
        num=num+1
        for valor in frases:
            #print('num:', num)
            #print('valor:', valor)
            #print('valor[1]:', valor[1])
            tokens_entidade = valor[0]
            #frase = valor[1]
            indices = valor[2]
            tipo_previsto=valor[4]
            #print('tokens_entidade:', tokens_entidade)
            #print('frase:', frase)
            #print('indices:', indices)
            #print('tipo_previsto:', tipo_previsto)
            entidades.append([tokens_entidade, indices, labels[tipo_previsto]])
            # ver se entidade está na dic_prediction
        frase = dic_predictions[num].copy()[0]
        dic_predictions_sentence[num] = [frase, entidades]
        entidades = list()
                     
    return dic_predictions_sentence

In [205]:
dic_predictions_sentence = getDicPredictionsSentence(combinacaoEntidadesAll, dic_predictions)
dic_predictions_sentence[20]

[[['Otimizo', 0],
  ['dose', 1],
  ['da', 2],
  ['sinvastatina', 3],
  ['para', 4],
  ['40mg', 5],
  ['/', 6],
  ['dia', 7],
  ['.', 8]],
 [['dose da sinvastatina para 40mg', [1, 2, 3, 4, 5], 'Tratamento'],
  ['sinvastatina', [3], 'O'],
  ['dose', [1], 'O'],
  ['dose da sinvastatina', [1, 2, 3], 'Tratamento'],
  ['da', [2], 'O'],
  ['para', [4], 'O'],
  ['sinvastatina para 40mg', [3, 4, 5], 'Tratamento']]]

In [207]:
# avaliar por regiao.. 
print('-----Avaliando modelo com filtro + downsampling (Região):-----')

region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_sentence, BATCH)

-----Avaliando modelo com filtro + downsampling (Região):-----
numErro1: 35
numErro2: 285
              precision    recall  f1-score   support

    Anatomia   0.815789  0.574074  0.673913        54
           O   0.747423  0.508772  0.605428       285
    Problema   0.530387  0.849558  0.653061       113
       Teste   0.742574  0.903614  0.815217        83
  Tratamento   0.658824  0.875000  0.751678        64

    accuracy                       0.672788       599
   macro avg   0.698999  0.742204  0.699859       599
weighted avg   0.702504  0.672788  0.665283       599



In [ ]:
print(confusion_matrix(region_true_list, region_pred_list))


In [208]:
# avaliar por regiao.. 
print('-----Avaliando modelo com filtro (Região):-----')
# para suprimir output
#model = ClassificationModel('bert', 'lisaterumi/sentence_pairs_nested_filtro', use_cuda=False)
model = ClassificationModel('bert', r'C:\Users\lisat\Downloads\sentece-filtro', use_cuda=False)
#combinacaoEntidadesAll = f.load_obj('combinacaoEntidadesAll_'+str(BATCH))
print('Chamando predict')

-----Avaliando modelo com filtro (Região):-----
Chamando predict


In [209]:
%%capture 
pred_region_labels = list()
for key, combinacao in enumerate(combinacaoEntidadesAll):
    if key<BATCH:    
        if len(combinacao)>0:
            lista = [l[0:2] for l in combinacao]
            predictions, _ = model.predict(lista) 
            pred_region_labels.append(predictions)
            for comb, label in zip(combinacao, predictions):
                comb.append(label)
f.save_obj('combinacaoEntidadesAllSentenceComLabel2_'+str(BATCH), combinacaoEntidadesAll)


In [210]:
combinacaoEntidadesAll[20]

[['dose da sinvastatina para 40mg',
  'Otimizo dose da sinvastatina para 40mg / dia .',
  [1, 2, 3, 4, 5],
  2,
  2,
  2],
 ['sinvastatina',
  'Otimizo dose da sinvastatina para 40mg / dia .',
  [3],
  0,
  0,
  0],
 ['dose', 'Otimizo dose da sinvastatina para 40mg / dia .', [1], 0, 0, 0],
 ['dose da sinvastatina',
  'Otimizo dose da sinvastatina para 40mg / dia .',
  [1, 2, 3],
  0,
  2,
  2],
 ['da', 'Otimizo dose da sinvastatina para 40mg / dia .', [2], 0, 0, 0],
 ['para', 'Otimizo dose da sinvastatina para 40mg / dia .', [4], 0, 0, 0],
 ['sinvastatina para 40mg',
  'Otimizo dose da sinvastatina para 40mg / dia .',
  [3, 4, 5],
  0,
  2,
  2]]

In [212]:
def getDicPredictionsSentence2(combinacaoEntidadesAll_pred, dic_predictions):
    labels = {0:'O', 1:'Problema', 2:'Tratamento', 3:'Teste', 4:'Anatomia'}
    num=-1
    dic_predictions_sentence={}
    entidades=list()
    numAcrescentou=0
    for frases in combinacaoEntidadesAll_pred:
        num=num+1
        for valor in frases:
            #print('num:', num)
            #print('valor:', valor)
            #print('valor[1]:', valor[1])
            tokens_entidade = valor[0]
            #frase = valor[1]
            indices = valor[2]
            tipo_previsto=valor[5]
            #print('tokens_entidade:', tokens_entidade)
            #print('frase:', frase)
            #print('indices:', indices)
            #print('tipo_previsto:', tipo_previsto)
            entidades.append([tokens_entidade, indices, labels[tipo_previsto]])
            # ver se entidade está na dic_prediction
        frase = dic_predictions[num].copy()[0]
        dic_predictions_sentence[num] = [frase, entidades]
        entidades = list()
                     
    return dic_predictions_sentence

In [213]:
dic_predictions_sentence = getDicPredictionsSentence2(combinacaoEntidadesAll, dic_predictions)
region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_sentence, BATCH)

numErro1: 35
numErro2: 285
              precision    recall  f1-score   support

    Anatomia   0.815789  0.574074  0.673913        54
           O   0.747423  0.508772  0.605428       285
    Problema   0.530387  0.849558  0.653061       113
       Teste   0.742574  0.903614  0.815217        83
  Tratamento   0.658824  0.875000  0.751678        64

    accuracy                       0.672788       599
   macro avg   0.698999  0.742204  0.699859       599
weighted avg   0.702504  0.672788  0.665283       599



In [214]:
print(confusion_matrix(region_true_list, region_pred_list))


[[ 31  17   4   2   0]
 [  6 145  81  24  29]
 [  0  17  96   0   0]
 [  1   7   0  75   0]
 [  0   8   0   0  56]]


In [215]:
region_true_list[:10]


['Problema',
 'Tratamento',
 'O',
 'Problema',
 'Problema',
 'Tratamento',
 'Tratamento',
 'Tratamento',
 'Tratamento',
 'O']

In [216]:
region_pred_list[:10]

['Problema',
 'Tratamento',
 'O',
 'Problema',
 'Problema',
 'Tratamento',
 'Tratamento',
 'Tratamento',
 'Tratamento',
 'Tratamento']

In [217]:
dicSentences_new_test[15]

[[['calcificação', 0, 1083],
  ['mitral', 1, 1096],
  ['e', 2, 1103],
  ['aórtica', 3, 1105],
  ['com', 4, 1113],
  ['refluxo', 5, 1117],
  ['leve', 6, 1125],
  ['.', 7, 1129]],
 [['calcificação mitral e aórtica com refluxo leve',
   [0, 1, 2, 3, 4, 5, 6],
   'Problema'],
  ['mitral', [1], 'Anatomia'],
  ['aórtica', [3], 'Anatomia']]]

In [218]:
dic_predictions[15]

[[['calcificação', 0],
  ['mitral', 1],
  ['e', 2],
  ['aórtica', 3],
  ['com', 4],
  ['refluxo', 5],
  ['leve', 6],
  ['.', 7]],
 [['calcificação', [0], 'Problema'],
  ['mitral', [1], 'Tratamento'],
  ['e', [2], 'Problema'],
  ['aórtica', [3], 'Anatomia'],
  ['refluxo leve', [5, 6], 'Problema']]]

In [219]:
dic_predictions_sentence[15]

[[['calcificação', 0],
  ['mitral', 1],
  ['e', 2],
  ['aórtica', 3],
  ['com', 4],
  ['refluxo', 5],
  ['leve', 6],
  ['.', 7]],
 [['calcificação', [0], 'Problema'],
  ['mitral', [1], 'Anatomia'],
  ['e', [2], 'O'],
  ['aórtica', [3], 'Anatomia'],
  ['refluxo leve', [5, 6], 'Problema'],
  ['refluxo', [5], 'Problema'],
  ['leve', [6], 'O']]]

In [220]:
# avaliar por regiao.. 
print('-----Avaliando modelo sem filtro (Região):-----')
# para suprimir output
model = ClassificationModel('bert', 'lisaterumi/sentence_pairs_nested_all', use_cuda=False)
#model = ClassificationModel('bert', r'C:\Users\lisat\Downloads\sentece-sem-filtro', use_cuda=False)
#combinacaoEntidadesAll = f.load_obj('combinacaoEntidadesAll_'+str(BATCH))
print('Chamando predict')
combinacaoEntidadesAll = f.load_obj('combinacaoEntidadesAllSentence_'+str(BATCH))


-----Avaliando modelo sem filtro (Região):-----


Downloading:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/560 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Chamando predict


In [221]:
%%capture 
pred_region_labels = list()
for key, combinacao in enumerate(combinacaoEntidadesAll):
    if key<BATCH:    
        if len(combinacao)>0:
            lista = [l[0:2] for l in combinacao]
            predictions, _ = model.predict(lista) 
            pred_region_labels.append(predictions)
            for comb, label in zip(combinacao, predictions):
                comb.append(label)
f.save_obj('combinacaoEntidadesAllSentenceComLabel3_'+str(BATCH), combinacaoEntidadesAll)


In [223]:
dic_predictions_sentence = getDicPredictionsSentence(combinacaoEntidadesAll, dic_predictions)
region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_sentence, BATCH)
print(confusion_matrix(region_true_list, region_pred_list))


numErro1: 35
numErro2: 285
              precision    recall  f1-score   support

    Anatomia   0.791667  0.703704  0.745098        54
           O   0.753695  0.536842  0.627049       285
    Problema   0.514970  0.761062  0.614286       113
       Teste   0.844444  0.915663  0.878613        83
  Tratamento   0.637363  0.906250  0.748387        64

    accuracy                       0.686144       599
   macro avg   0.708428  0.764704  0.722687       599
weighted avg   0.712228  0.686144  0.683106       599

[[ 38  12   4   0   0]
 [ 10 153  77  12  33]
 [  0  26  86   1   0]
 [  0   7   0  76   0]
 [  0   5   0   1  58]]


### Agora, merge com dic_predictions?/

Acho que nao precisa!

### Agora, com pos processamento

Regras:

- Pega os de dentro (aninhadas), apenas se o tipo for diferente do tipo da entidade de fora... se for do mesmo tipo, não pega...

In [224]:
def isEntidadeAninhada(a, b):
    inicio1=a[0]
    fim1=a[-1]
    #print(inicio1, fim1)
    inicio2=b[0]
    fim2=b[-1]
    #print(inicio2, fim2)
    if inicio1 >=inicio2 and fim1 <=fim2:
        return True
    else:
        return False
isEntidadeAninhada([2], [2, 3])

True

In [225]:
# juntar all_predictions
print('juntando sentence + ner, com pos processamento')
#dic_predictions = f.load_obj('dic_predictions_results_ner_'+str(BATCH))
#dic_predictions_all = f.getDicPredictionsAll(combinacaoEntidadesAll_pred, dic_predictions)

def juntaPredictons(dic_predictions, dic_predictions_sentence):
    dic_predictions_all = {}
    for key, value in dic_predictions.items():
        #dic_predictions_all[key] = value.copy()
        tokens = value[0].copy()
        listaEntidadesNer = value[1].copy()
        listaIndicesNer=list()
        listaTipoNer=list()
        for entidadeNer in listaEntidadesNer:
            listaIndicesNer.append(entidadeNer[1])
            listaTipoNer.append(entidadeNer[2])
        #print('listaIndicesNer:', listaIndicesNer)
        listaEntidadesSpan = dic_predictions_sentence[key][1]
        for entidadeSpan in listaEntidadesSpan:
            indices = entidadeSpan[1]
            if indices in listaIndicesNer:
                #print('ja tem, pulando')
                pass
            else:
                #print('incluindo:', entidadeSpan)
                # só acrescenta se nao tem entidade mais externa do mesmo tipo
                # ver pelo indice
                isAninhada=False
                tipo=entidadeSpan[2]
                for indicesNer, tipoNer in zip(listaIndicesNer, listaTipoNer):
                    isAninhada = isEntidadeAninhada(indices, indicesNer) # é aninhada com alguma entidade?
                    if isAninhada and tipoNer == tipo:
                        break
                # ver pelo tipo tb
                if not isAninhada:
                    print('Não é aninhada, incluindo:', entidadeSpan)
                    listaEntidadesNer.append(entidadeSpan)
                else:
                    #print('aninhada:', entidadeSpan)
                    pass
        dic_predictions_all[key] = [tokens, listaEntidadesNer]
        #if key>15:
        #    break
    return dic_predictions_all

dic_predictions_all = juntaPredictons(dic_predictions, dic_predictions_sentence)
#dic_predictions_all
region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_all, BATCH)

juntando sentence + ner, com pos processamento
Não é aninhada, incluindo: ['glicazida', [22], 'O']
Não é aninhada, incluindo: ['metformina', [9], 'O']
Não é aninhada, incluindo: ['carvedilol', [20], 'Problema']
Não é aninhada, incluindo: ['esquerdo', [3], 'O']
Não é aninhada, incluindo: ['ventrículo', [2], 'O']
Não é aninhada, incluindo: ['VE', [36], 'Anatomia']
Não é aninhada, incluindo: ['moderado', [38], 'O']
Não é aninhada, incluindo: ['difuso', [34], 'O']
Não é aninhada, incluindo: ['grau moderado', [37, 38], 'O']
Não é aninhada, incluindo: ['do', [35], 'O']
Não é aninhada, incluindo: ['difuso do VE', [34, 35, 36], 'O']
Não é aninhada, incluindo: ['insuf', [40], 'O']
Não é aninhada, incluindo: ['insuf', [44], 'O']
Não é aninhada, incluindo: ['VE grau', [36, 37], 'O']
Não é aninhada, incluindo: ['intermitente', [3], 'O']
Não é aninhada, incluindo: ['MMII', [5], 'Anatomia']
Não é aninhada, incluindo: ['turvação', [11], 'O']
Não é aninhada, incluindo: ['visual', [12], 'O']
Não é anin

In [229]:
region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_all, BATCH)
print(confusion_matrix(region_true_list, region_pred_list))


numErro1: 66
numErro2: 130
              precision    recall  f1-score   support

    Anatomia   0.766667  0.425926  0.547619        54
           O   0.561290  0.669231  0.610526       130
    Problema   0.845455  0.823009  0.834081       113
       Teste   0.870588  0.891566  0.880952        83
  Tratamento   0.828125  0.828125  0.828125        64

    accuracy                       0.743243       444
   macro avg   0.774425  0.727571  0.740261       444
weighted avg   0.754871  0.743243  0.741689       444

[[23 28  1  1  1]
 [ 7 87 16 10 10]
 [ 0 20 93  0  0]
 [ 0  9  0 74  0]
 [ 0 11  0  0 53]]


In [226]:
dic_predictions_all[20]

[[['Otimizo', 0],
  ['dose', 1],
  ['da', 2],
  ['sinvastatina', 3],
  ['para', 4],
  ['40mg', 5],
  ['/', 6],
  ['dia', 7],
  ['.', 8]],
 [['dose da sinvastatina para 40mg', [1, 2, 3, 4, 5], 'Tratamento']]]

In [227]:
dic_predictions_sentence[20]

[[['Otimizo', 0],
  ['dose', 1],
  ['da', 2],
  ['sinvastatina', 3],
  ['para', 4],
  ['40mg', 5],
  ['/', 6],
  ['dia', 7],
  ['.', 8]],
 [['dose da sinvastatina para 40mg', [1, 2, 3, 4, 5], 'Tratamento'],
  ['sinvastatina', [3], 'O'],
  ['dose', [1], 'O'],
  ['dose da sinvastatina', [1, 2, 3], 'O'],
  ['da', [2], 'O'],
  ['para', [4], 'O'],
  ['sinvastatina para 40mg', [3, 4, 5], 'Tratamento']]]

In [228]:
dic_predictions[20]

[[['Otimizo', 0],
  ['dose', 1],
  ['da', 2],
  ['sinvastatina', 3],
  ['para', 4],
  ['40mg', 5],
  ['/', 6],
  ['dia', 7],
  ['.', 8]],
 [['dose da sinvastatina para 40mg', [1, 2, 3, 4, 5], 'Tratamento']]]